# Setup Drive

In [ ]:
import numpy as np
import os
import scipy.sparse as ss
from scipy.stats import laplace
from scipy.stats import norm as gaussian
from scipy.optimize import lsq_linear

from google.colab import drive
drive.mount("/content/drive")
datadir = "/content/drive/MyDrive/bu_dp_data"
if not os.path.exists(datadir):
    os.mkdir(datadir)


Mounted at /content/drive


# Mechanisms


In [ ]:
def l2_sensitivity(mymatrix):
    x = mymatrix if type(mymatrix) == type(np.array([0])) else mymatrix.todense()
    sens=np.sqrt(np.multiply(x,x).sum(axis=0).max())
    return sens

def l1_sensitivity(mymatrix):
    x = mymatrix if type(mymatrix) == type(np.array([0])) else mymatrix.todense()    
    sens=np.abs(x).sum(axis=0).max()
    return sens

def answer_query(query, data):
    if type(query) == type(np.array([0])):
      return query @ data # matrix multiplication in numpy
    else:
      return query * data # matrix multiplication in sparse matrices

def laplace_mechanism(epsilon, query, data):
    sens = l1_sensitivity(query)
    scale = sens/epsilon
    the_answer = answer_query(query, data)
    return the_answer + laplace.rvs(scale=scale, size=the_answer.shape)

def gaussian_mechanism(rho, query, data):
    sens = l2_sensitivity(query)
    scale = sens/np.sqrt(2*rho)
    the_answer = answer_query(query, data)
    return the_answer + gaussian.rvs(scale=scale, size=the_answer.shape)


def postprocess(query, noisy_answer, nonneg=False):
    """ Given a noisy answer to the query, estimate x using least squares and 
    optionally enforce nonnegativity
    """
    if nonneg:
        bounds = (0, np.inf)
    else:
        bounds = (-np.inf, np.inf)
    solution_object = lsq_linear(query, noisy_answer, bounds=bounds)
    return solution_object.x





# Code for running simulations


In [ ]:
from ipywidgets import IntProgress
from IPython.display import display

def run_simulation(data, query, fname, amount=2000, mechanism=laplace_mechanism, param=0.25, savepoint = 1000, progress_interval=100):
    """ Run query and postprocess simulations, returns their results
    
    Input:
        data:  a data vector
        query:  a set of queries represented as a matrix. query times data are the answers
        fname: the base name of a file that will store the results of the simulation
                set fname to None or "" to avoid any saving
        amount: number of simulations to perform. It will read in any saved simulations and perform the remaining ones
        mechanism: the function that provide the privacy protections (e.g,. laplace_mechanism or gaussian_mechanism)
                mechanism is a funciton whose inputs are a privacy parameter, query, and data
        param: privacy parameter for the mechanism
        savepoint: save to google drive every `savepoint' simulations
        progress_interval: update the progress bar every `progress_interval' simulations

    output:
        a list of noisy query answers
        a list of ols postprocessed estimates of the data
        a list of nnls postprocessed estimates of the data
    """
    n = data.size

    # see if loaded data exists
    noisy_file = f"{datadir}/{fname}_{n}_{mechanism.__name__}_{param}_na.npy"
    ols_file = f"{datadir}/{fname}_{n}_{mechanism.__name__}_{param}_ols.npy"
    nnls_file = f"{datadir}/{fname}_{n}_{mechanism.__name__}_{param}_nnls.npy"
    if os.path.exists(noisy_file):
        noisy_arr = list(np.load(noisy_file))
        ols_arr = list(np.load(ols_file))
        nnls_arr = list(np.load(nnls_file))
        #print("loading files ...")
        #print("loaded", len(noisy_arr), "simulations")
    else:
        noisy_arr = []
        ols_arr = []
        nnls_arr = []
    assert len(noisy_arr) == len(ols_arr) == len(nnls_arr)

    starting = len(noisy_arr)
    remaining = max(0, amount - len(noisy_arr))
    #print(remaining, "simulations remain")
    progress_bar = IntProgress(min=0, max=max(starting, amount))
    display(progress_bar)
    progress_bar.value = starting

    for i in range(starting, amount):
        noisy_answer = mechanism(param, query, data)
        ols_solution = postprocess(query, noisy_answer, nonneg=False)
        nnls_solution = postprocess(query, noisy_answer, nonneg=True)
        noisy_arr.append(noisy_answer)
        ols_arr.append(ols_solution)
        nnls_arr.append(nnls_solution)
        if i > 0 and i % savepoint == 0 and fname:
            np.save(noisy_file, noisy_arr)
            np.save(ols_file, ols_arr)
            np.save(nnls_file, nnls_arr)
        if i % progress_interval == 0:
            progress_bar.value = i
    progress_bar.value=max(starting, amount)
    if fname:
        np.save(noisy_file, noisy_arr)
        np.save(ols_file, ols_arr)
        np.save(nnls_file, nnls_arr)
    return (noisy_arr, ols_arr, nnls_arr)

    


# Minimizing sum squared error



In [ ]:
def experiment_ssq():
    amount = 3000 # number of simulations
    rho = 1
    data = np.array([100, 200, 300, 400, 500])
    workload_query = np.array([[1, 1, 1, 1, 1],
                               [1, 0, 0, 0, 0],
                               [0, 1, 0, 0, 0],
                               [0, 0, 1, 0, 0],
                               [0, 0, 0, 1, 0],
                               [0, 0, 0, 0, 1]])
    ###########################################################
    # change this strategy_query (e.g, rescale rows, add rows)
    ###########################################################
    strategy_query = np.array([[1, 1, 1, 1, 1],
                               [1, 0, 0, 0, 0],
                               [0, 1, 0, 0, 0],
                               [0, 0, 1, 0, 0],
                               [0, 0, 0, 1, 0],
                               [0, 0, 0, 0, 1]])
    ###########################################################
    (noisy_arr, ols_arr, nnls_arr) = run_simulation(data, strategy_query, fname=None, amount=amount, mechanism=gaussian_mechanism, param=rho)
    true_answer = answer_query(workload_query, data)
    ols_error = sum([((answer_query(workload_query, ans) - true_answer)**2).sum() for ans in ols_arr])/amount
    print(f"Your estimated error: {ols_error}")
    # compute optimal error:
    betasq = 2 * rho
    d = 5
    a = (-3 + d) / (-1+d - np.sqrt(1+d)) / betasq
    b = (-3 + d)*(2 - np.sqrt(1+d)) / (-1+d - np.sqrt(1+d)) / (-1-d + (-1+d)*np.sqrt(1+d)) / betasq
    optimal_cost = d*(a+b) + d*a + d*d*b
    print(f"Optimal sum squared error is {optimal_cost}")

experiment_ssq()


IntProgress(value=0, max=3000)

Your estimated error: 5.034730326976701
Optimal sum squared error is 4.159591794226543


# Hitting Error Targets

In [ ]:
def experiment_target():
    amount = 3000 # number of simulations
    rho = 1
    data = np.array([100, 200, 300, 400, 500])
    workload_query = np.array([[1, 1, 1, 1, 1],
                               [1, 0, 0, 0, 0],
                               [0, 1, 0, 0, 0],
                               [0, 0, 1, 0, 0],
                               [0, 0, 0, 1, 0],
                               [0, 0, 0, 0, 1]])
    ###########################################################
    # change this strategy_query (e.g, rescale rows, add rows)
    ###########################################################
    strategy_query = np.array([[1, 1, 1, 1, 1],
                               [1, 0, 0, 0, 0],
                               [0, 1, 0, 0, 0],
                               [0, 0, 1, 0, 0],
                               [0, 0, 0, 1, 0],
                               [0, 0, 0, 0, 1]])
    ###########################################################
    (noisy_arr, ols_arr, nnls_arr) = run_simulation(data, strategy_query, fname=None, amount=amount, mechanism=gaussian_mechanism, param=rho)
    true_answer = answer_query(workload_query, data)
    ols_error = sum([((answer_query(workload_query, ans) - true_answer)**2) for ans in ols_arr])/amount
    print(f"Your estimated errors: {ols_error}")
    # compute optimal error:
    betasq = 2 * rho
    d = 5
    gamma = 2*d / (1+d) / betasq
    optimal_cost = gamma
    print(f"Optimal target for each query is {optimal_cost}")
    print(f"Targets exceeded by a factor of {ols_error.max()/gamma}")

experiment_target()


IntProgress(value=0, max=3000)

Your estimated errors: [0.85911868 0.79393816 0.7850811  0.84558753 0.81335281 0.86724989]
Optimal target for each query is 0.8333333333333334
Targets exceeded by a factor of 1.0406998675639156


# NNLS and OLS Comparisons



In [ ]:
def identity_sum_query(n=1000):
   """ returns a matrix like:
   1 1 1 1
   1 0 0 0
   0 1 0 0 
   0 0 1 0
   0 0 0 1 where n is the number of columns"""
   mat = np.vstack([np.ones(n), np.eye(n)])
   return ss.csr_matrix(mat)

def sample_data(n=1000, first=50):
   data = np.zeros(n)
   data[0]=50
   return data

def alternate_data(n=1000, first=50):
   data = np.ones(n)
   data[0]=50
   return data



Questions to consider:


1.   What utility measures should we use?
2.   How do answers obtained from the OLS estimate of the data compare to the direct noisy answer?
3.   How does NNLS accuracy compare to OLS accuracy? Are there any metrics where NNLS is better? Are there any metrics where OLS is better?





In [ ]:
n = 1000
data = sample_data(n)
query = identity_sum_query(n)
(noisy_arr, ols_arr, nnls_arr) = run_simulation(data, query, "id_sum_lap25")

data2 = alternate_data(n)
(noisy_arr2, ols_arr2, nnls_arr2) = run_simulation(data2, query, "alt_id_sum_lap25")


# noisy_arr[0] is a numpy array of noisy query answers for the first simulation
# ols_arr[0] is an OLS estiamte of the data based on the noisy answers noisy_arr[0]
# nnls_arr[0] is the corresponding nnls estimate
# answer_query(query, ols_arr[0]) is the answer to the queries obtained from the first OLS estimate



IntProgress(value=0, max=2000)

IntProgress(value=0, max=2000)

These are the error measures we are coding up

In [ ]:
# First error measure

def mean_squared_error(query, data, synth_arr):
    true_answers = answer_query(query, data)
    amount = len(synth_arr)
    return sum([np.sum((answer_query(query, synth)-true_answers)**2) for synth in synth_arr]) / amount

# second error measure
def max_outlier_error(query, data, synth_arr):
    true_answers = answer_query(query, data)
    amount = len(synth_arr)
    return sum([np.max((answer_query(query, synth) - true_answers)**2) for synth in synth_arr]) / amount

# third error measure
def max_expected_error(query, data, synth_arr):
    true_answers = answer_query(query, data)
    amount = len(synth_arr)
    return max(sum([(answer_query(query, synth) - true_answers)**2 for synth in synth_arr]) / amount)

# fourth error measure
def max_bias_error(query, data, synth_arr):
    true_answers = answer_query(query, data)
    amount = len(synth_arr)
    return max(np.abs(sum([answer_query(query, synth) - true_answers for synth in synth_arr])) / amount)

# 5th error measure
def look_at_three(query, data, synth_arr):
    true_answers = answer_query(query, data)
    amount = len(synth_arr)
    first_three = sum([((answer_query(query, synth) - true_answers)**2)[0:3] for synth in synth_arr]) / amount
    for x in first_three:
         print(x)

# 6th error measure
def look_at_three_bias(query, data, synth_arr):
    true_answers = answer_query(query, data)
    amount = len(synth_arr)
    first_three = sum([(answer_query(query, synth) - true_answers)[0:3] for synth in synth_arr]) / amount
    for x in first_three:
         print(x)



The results:

In [ ]:
print("OLS Sum Squared Error", mean_squared_error(query, data, ols_arr)) 
print("NNLS Sum Squared Error", mean_squared_error(query, data, nnls_arr))

print("OLS outlier error", max_outlier_error(query, data, ols_arr))
print("NNLS outlier error", max_outlier_error(query, data, nnls_arr))

print("OLS max expected error", max_expected_error(query, data, ols_arr))
print("NNLS max expected error", max_expected_error(query, data, nnls_arr))

print("OLS max bias error", max_bias_error(query, data, ols_arr))
print("NNLS max bias error", max_bias_error(query, data, nnls_arr))

print("scale for comparison is ", np.sqrt(32))

print("OLS look at 3")
look_at_three(query, data, ols_arr)
print("NNLS look at 3")
look_at_three(query, data, nnls_arr)

print("OLS look at 3 bias")
look_at_three_bias(query, data, ols_arr)
print("NNLS look at 3 bias")
look_at_three_bias(query, data, nnls_arr)





OLS Sum Squared Error 127493.28808520139
NNLS Sum Squared Error 3228.041737320452
OLS outlier error 3620.0334912175504
NNLS outlier error 1540.6200665928957
OLS max expected error 147.82950425495594
NNLS max expected error 1170.419725889349
OLS max bias error 0.9207050375489471
NNLS max bias error 32.477738599577336
scale for comparison is  5.656854249492381
OLS look at 3
127.87008443666997
123.3760430728707
123.20262285502436
NNLS look at 3
1170.419725889349
1140.625549791372
0.3890884157590231
OLS look at 3 bias
-0.3815311266145748
0.21942292943286607
0.23852710365686444
NNLS look at 3 bias
32.477738599577336
-32.3062635879303
0.03863505106343246


In [ ]:
# Measuring error

print("OLS Sum Squared Error", mean_squared_error(query, data2, ols_arr2)) 
print("NNLS Sum Squared Error", mean_squared_error(query, data2, nnls_arr2))

print("OLS outlier error", max_outlier_error(query, data2, ols_arr2))
print("NNLS outlier error", max_outlier_error(query, data2, nnls_arr2))

print("OLS max expected error", max_expected_error(query, data2, ols_arr2))
print("NNLS max expected error", max_expected_error(query, data2, nnls_arr2))

print("OLS max bias error", max_bias_error(query, data2, ols_arr2))
print("NNLS max bias error", max_bias_error(query, data2, nnls_arr2))

print("scale for comparison is ", np.sqrt(32))

print("OLS look at 3")
look_at_three(query, data2, ols_arr2)
print("NNLS look at 3")
look_at_three(query, data2, nnls_arr2)

print("OLS look at 3 bias")
look_at_three_bias(query, data2, ols_arr2)
print("NNLS look at 3 bias")
look_at_three_bias(query, data2, nnls_arr2)



OLS Sum Squared Error 127807.94237388848
NNLS Sum Squared Error 15733.100985631098
OLS outlier error 3687.4332614820687
NNLS outlier error 1944.907179529078
OLS max expected error 146.63053477635512
NNLS max expected error 268.3611301503197
OLS max bias error 0.7475095526152166
NNLS max bias error 12.152800628834381
scale for comparison is  5.656854249492381
OLS look at 3
121.23601239672138
127.1880916170575
122.30989824087762
NNLS look at 3
268.3611301503197
266.1891720118926
15.794787033038672
OLS look at 3 bias
0.27306245471356466
-0.09736579269559009
0.4149287398135996
NNLS look at 3 bias
12.152800628834381
-11.944933338489925
0.034642179364533676
